# Lista 07

## Nome: Jorge Lucas Colares Martins
## Matricula: 402119

## Importando bibliotecas

In [226]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import model_selection

from sklearn import pipeline

## Importando Dataframe

In [185]:
df = pd.read_csv("diabetes.csv",delimiter=',')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


## Divindo em treino e teste

In [186]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Outcome',axis=1), df['Outcome'], test_size=0.2, random_state= 42, stratify=df['Outcome'])

## Divindo em train_holdout e val_holdout

In [187]:
X_train_holdout, X_val_holdout, y_train_holdout, y_val_holdout = train_test_split(X_train, y_train, test_size=0.3, random_state= 42, stratify=y_train)

## Criando range de K (do KNN)

In [188]:
k_range = np.arange(1,101)
print(k_range)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]


## Questão 01

### Criando lista para armazenar os treinos de cada K (KNN) e Treinando

In [189]:
holdout_train_list = []

for i in k_range:
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train_holdout, y_train_holdout)
    holdout_train_list.append(knn)


### Validando e guardando resultados

In [190]:
holdout_val_list = []
j=0
for i in holdout_train_list:
    y_pred_val = i.predict(X_val_holdout)
    holdout_val_list.append([metrics.f1_score(y_val_holdout, y_pred_val, average='weighted', zero_division=0), k_range[j]])
    j= j+1

holdout_val_list.sort(reverse=True)

In [191]:
print(*holdout_val_list, sep = "\n")

[0.8098258905806076, 51]
[0.8098258905806076, 49]
[0.8083248078448253, 50]
[0.8067310418372291, 63]
[0.8000160160160161, 52]
[0.795017771645546, 61]
[0.7948146619902345, 47]
[0.7933197348291688, 42]
[0.7932239518955386, 56]
[0.7900434235816151, 48]
[0.7898485008260396, 39]
[0.7883586226502256, 43]
[0.7882522522522524, 54]
[0.7867744833782571, 57]
[0.7863518380325104, 62]
[0.7863518380325104, 60]
[0.7850914512199558, 53]
[0.784898884898885, 41]
[0.7834154765452476, 45]
[0.7834154765452476, 31]
[0.7818374978752337, 34]
[0.7818374978752337, 32]
[0.7814081777181408, 67]
[0.78016038016038, 55]
[0.7769185481988864, 37]
[0.7769185481988864, 36]
[0.7769185481988864, 35]
[0.7769185481988864, 33]
[0.7769185481988864, 23]
[0.7752487797770817, 44]
[0.7752487797770817, 40]
[0.7744824957009832, 65]
[0.7734747729075208, 46]
[0.7715912312621799, 58]
[0.7703552609212987, 25]
[0.7703552609212987, 21]
[0.7695924035407429, 71]
[0.7695924035407429, 69]
[0.7685898738530318, 59]
[0.7685898738530318, 38]
[0.7

### Usando melhor K para traino e teste normal

In [192]:
knn = KNeighborsClassifier(n_neighbors=holdout_val_list[0][1])
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=51)

### Exibindo resultado

In [193]:
y_pred = knn.predict(X_test)
print(metrics.classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.72      0.81      0.76       100
           1       0.55      0.43      0.48        54

    accuracy                           0.68       154
   macro avg       0.64      0.62      0.62       154
weighted avg       0.66      0.68      0.66       154



### Usando escalonamento

In [194]:
pipe_holdout_list = []

for i in k_range:
    pipe_holdout = Pipeline(steps = [('scale',StandardScaler()),('KNN',KNeighborsClassifier(n_neighbors=i))])
    pipe_holdout_list.append(pipe_holdout.fit(X_train_holdout,y_train_holdout))

### Mostrando resultados

In [195]:
result_holdout_pipe = []
j=0

for i in pipe_holdout_list:
    holdout_pred = i.predict(X_val_holdout)
    result_holdout_pipe.append([metrics.f1_score(y_val_holdout, holdout_pred, average='weighted', zero_division=0), k_range[j]])
    j= j+1

result_holdout_pipe.sort(reverse=True)

In [196]:
print(*result_holdout_pipe, sep = "\n")

[0.7573967790184007, 19]
[0.7555475266815472, 37]
[0.7550506608452175, 29]
[0.7525485618508875, 17]
[0.7483921878082461, 51]
[0.7481646908788137, 25]
[0.7481646908788137, 23]
[0.7477108319365817, 15]
[0.7461129082750705, 11]
[0.7459608551859475, 33]
[0.7459608551859475, 31]
[0.745448861238335, 21]
[0.7433864065997517, 8]
[0.7432542332761288, 59]
[0.742598324844869, 16]
[0.7411436699857753, 42]
[0.7411436699857753, 39]
[0.7411436699857753, 36]
[0.7388744687064015, 55]
[0.7388744687064015, 35]
[0.7388744687064015, 27]
[0.7385146413445003, 46]
[0.7380622566300316, 9]
[0.7364108634181626, 45]
[0.7364108634181626, 41]
[0.7364108634181626, 30]
[0.7357270066947488, 73]
[0.7338783549309865, 14]
[0.7338783549309865, 12]
[0.7337983682109491, 53]
[0.7337983682109491, 49]
[0.7337983682109491, 48]
[0.7337983682109491, 47]
[0.7337983682109491, 44]
[0.7337983682109491, 40]
[0.7316985048943813, 43]
[0.7316985048943813, 34]
[0.7311582503935299, 18]
[0.728442154529111, 7]
[0.7280909480909481, 58]
[0.726

### Usando melhor resultado do pipe de K (KNN)

In [197]:
knn = KNeighborsClassifier(n_neighbors=result_holdout_pipe[0][1])
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=19)

### Exibindo resultado

In [198]:
y_pred = knn.predict(X_test)
print(metrics.classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.77      0.83      0.80       100
           1       0.63      0.54      0.58        54

    accuracy                           0.73       154
   macro avg       0.70      0.68      0.69       154
weighted avg       0.72      0.73      0.72       154



## Questão 02

In [199]:
knn_list = []

for i in k_range:
    knn = KNeighborsClassifier(n_neighbors=i)
    knn_list.append(knn)

cv_list = []
j = 0
for i in knn_list:
    cv_list.append([cross_val_score(i,X_train,y_train,cv=5,scoring='f1_weighted').mean(),k_range[j]])
    j =j+1

### Exibindo resultados

In [200]:
cv_list.sort(reverse=True)
print(*cv_list, sep = "\n")

[0.7606241388532532, 12]
[0.758260766571087, 14]
[0.7564372883827548, 15]
[0.7503549799350728, 11]
[0.7500613027439541, 17]
[0.7492766940816857, 16]
[0.7488404793577346, 10]
[0.747555881225926, 19]
[0.7448849237389392, 27]
[0.7436937094136458, 13]
[0.7432099292202186, 9]
[0.7428826323389677, 29]
[0.7405528023366097, 36]
[0.7399650411141376, 23]
[0.7395763029378862, 28]
[0.7392051294083578, 54]
[0.7391921881495435, 25]
[0.7384521433641553, 26]
[0.7374244968245739, 20]
[0.7371472480935382, 21]
[0.7367600481660022, 52]
[0.7366779405819885, 35]
[0.7357486409159828, 63]
[0.7354765596511476, 34]
[0.7353565440374894, 53]
[0.7353416764765986, 33]
[0.7352273454313959, 31]
[0.7348283833554563, 37]
[0.7346699681169848, 49]
[0.7346491747446551, 5]
[0.7341867319912121, 59]
[0.7340963140905447, 24]
[0.7337688717326969, 22]
[0.7336859157550202, 41]
[0.7335790612858796, 39]
[0.7328788922160616, 32]
[0.7328777726837418, 50]
[0.7326505537363621, 18]
[0.7326001474005042, 55]
[0.7323084867542471, 43]
[0.7

### Usando o melhor resultado

In [201]:
knn = KNeighborsClassifier(n_neighbors=cv_list[0][1])
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=12)

### Exibindo resultado

In [202]:
y_pred = knn.predict(X_test)
print(metrics.classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.74      0.84      0.79       100
           1       0.60      0.44      0.51        54

    accuracy                           0.70       154
   macro avg       0.67      0.64      0.65       154
weighted avg       0.69      0.70      0.69       154



### Com escalonamento

In [203]:
esc_list = []

for i in k_range:
    esc_list.append(Pipeline(steps = [('scale',StandardScaler()),('KNN',KNeighborsClassifier(n_neighbors=i))]))

cv_esc_list = []
j = 0
for i in esc_list:
    cv_esc_list.append([cross_val_score(i,X_train,y_train,cv=5,scoring='f1_weighted').mean(),k_range[j]])
    j =j+1

In [204]:
cv_esc_list.sort(reverse=True)
print(*cv_esc_list, sep = "\n")

[0.7507420007654881, 31]
[0.7485312592517452, 30]
[0.7473043240771392, 13]
[0.7467381782824891, 29]
[0.7465483105942291, 41]
[0.7449011096556681, 35]
[0.7448486457570255, 33]
[0.744209984300043, 15]
[0.7412629069950082, 14]
[0.7401998752328218, 24]
[0.739968342864092, 25]
[0.7394140965024014, 7]
[0.7392839849411941, 9]
[0.7390136639665208, 5]
[0.7388287629854672, 45]
[0.738820473041174, 32]
[0.7380539193535964, 42]
[0.7380397871255525, 37]
[0.7367209127258352, 11]
[0.7353603495110055, 39]
[0.7349044065501291, 17]
[0.7348863779724158, 20]
[0.7347352032580259, 23]
[0.7346151154816634, 49]
[0.734143346654964, 10]
[0.7339137258082209, 40]
[0.733848984933099, 43]
[0.7338093400301827, 21]
[0.7336668652908517, 55]
[0.7334698091384105, 19]
[0.7333589731099172, 56]
[0.7330100206087724, 16]
[0.732764780081105, 22]
[0.7324137413531135, 12]
[0.7316924690116421, 28]
[0.7311915591763735, 47]
[0.7307722544085318, 34]
[0.7307175358299002, 18]
[0.730629798131661, 26]
[0.7305396329098646, 27]
[0.7305327

### Usando melhor resultado

In [205]:
knn = KNeighborsClassifier(n_neighbors=cv_esc_list[0][1])
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=31)

### Exibindo resultado

In [206]:
y_pred = knn.predict(X_test)
print(metrics.classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.72      0.81      0.76       100
           1       0.54      0.41      0.46        54

    accuracy                           0.67       154
   macro avg       0.63      0.61      0.61       154
weighted avg       0.65      0.67      0.66       154



## Questão 03

### Usando GridSearchCV

In [214]:
knn = KNeighborsClassifier()

param_grid = [{'n_neighbors': k_range}]

In [216]:
gs = model_selection.GridSearchCV(estimator=knn, param_grid=param_grid, 
                    return_train_score=False, scoring='f1_weighted', cv=5)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.7606241388532532
{'n_neighbors': 12}


### Exibindo novo DF

In [222]:
df1 = pd.DataFrame(gs.cv_results_)
df1 = df1.sort_values(by='rank_test_score')
df1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,0.003601,0.000800,0.009402,0.000800,12,{'n_neighbors': 12},0.754521,0.776571,0.729023,0.736149,0.806857,0.760624,0.028379,1
13,0.005801,0.001600,0.013203,0.001833,14,{'n_neighbors': 14},0.752093,0.793758,0.690636,0.732693,0.822124,0.758261,0.046051,2
14,0.006202,0.000749,0.013003,0.000632,15,{'n_neighbors': 15},0.764077,0.797780,0.679818,0.732146,0.808366,0.756437,0.046743,3
10,0.003801,0.001167,0.009603,0.001021,11,{'n_neighbors': 11},0.734783,0.801087,0.700802,0.721930,0.793173,0.750355,0.039782,4
16,0.006201,0.001940,0.012403,0.002059,17,{'n_neighbors': 17},0.761841,0.775363,0.700802,0.711541,0.800760,0.750061,0.038104,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,0.003600,0.000489,0.011203,0.001167,88,{'n_neighbors': 88},0.717800,0.688104,0.661411,0.674702,0.743554,0.697114,0.029809,96
0,0.004401,0.000801,0.010202,0.002136,1,{'n_neighbors': 1},0.647306,0.644077,0.686524,0.723577,0.779294,0.696156,0.050706,97
95,0.003601,0.000490,0.009732,0.000743,96,{'n_neighbors': 96},0.754213,0.657870,0.668045,0.657870,0.733000,0.694199,0.041063,98
99,0.003995,0.000633,0.010029,0.001677,100,{'n_neighbors': 100},0.743144,0.669757,0.669757,0.639491,0.747382,0.693906,0.043385,99


### Usando melhor resultado

In [224]:
best_k = df1['param_n_neighbors'].iloc[0]

knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=12)

### Exibindo resultado

In [225]:
y_pred = knn.predict(X_test)
print(metrics.classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.74      0.84      0.79       100
           1       0.60      0.44      0.51        54

    accuracy                           0.70       154
   macro avg       0.67      0.64      0.65       154
weighted avg       0.69      0.70      0.69       154



### Com escalonamento

In [229]:
knn_esc = pipeline.make_pipeline(StandardScaler(),KNeighborsClassifier())

param_grid = [{'kneighborsclassifier__n_neighbors': k_range}]

gs = model_selection.GridSearchCV(estimator=knn_esc, param_grid=param_grid, 
                    return_train_score=False, scoring='f1_weighted', cv=5)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.7507420007654881
{'kneighborsclassifier__n_neighbors': 31}


### Exibindo novo DF

In [230]:
df2 = pd.DataFrame(gs.cv_results_)
df2 = df2.sort_values(by='rank_test_score')
df2

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kneighborsclassifier__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
30,0.006401,0.001357,0.010402,0.001357,31,{'kneighborsclassifier__n_neighbors': 31},0.732146,0.793758,0.732146,0.728918,0.766743,0.750742,0.025590,1
29,0.006601,0.000800,0.011603,0.001855,30,{'kneighborsclassifier__n_neighbors': 30},0.736149,0.791448,0.729023,0.724797,0.761240,0.748531,0.024902,2
12,0.010003,0.003225,0.014202,0.002713,13,{'kneighborsclassifier__n_neighbors': 13},0.717899,0.764077,0.754521,0.736149,0.763877,0.747304,0.017873,3
28,0.008002,0.002098,0.012403,0.001855,29,{'kneighborsclassifier__n_neighbors': 29},0.707823,0.791448,0.749427,0.710846,0.774147,0.746738,0.033347,4
40,0.007201,0.002136,0.012003,0.001674,41,{'kneighborsclassifier__n_neighbors': 41},0.761013,0.756689,0.703932,0.746990,0.764118,0.746548,0.022078,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,0.004805,0.000398,0.012201,0.000746,94,{'kneighborsclassifier__n_neighbors': 94},0.701652,0.690690,0.690690,0.625263,0.697247,0.681109,0.028230,96
97,0.011003,0.004605,0.015603,0.002059,98,{'kneighborsclassifier__n_neighbors': 98},0.689687,0.690690,0.690690,0.631255,0.697247,0.679914,0.024478,97
99,0.008002,0.002099,0.017604,0.002577,100,{'kneighborsclassifier__n_neighbors': 100},0.689687,0.690690,0.690690,0.631255,0.697247,0.679914,0.024478,97
91,0.005001,0.000632,0.012203,0.001167,92,{'kneighborsclassifier__n_neighbors': 92},0.689687,0.690690,0.703190,0.625263,0.690477,0.679862,0.027755,99


In [231]:
best_k = df2['param_kneighborsclassifier__n_neighbors'].iloc[0]

knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=31)

### Exibindo resultados

In [232]:
y_pred = knn.predict(X_test)
print(metrics.classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.72      0.81      0.76       100
           1       0.54      0.41      0.46        54

    accuracy                           0.67       154
   macro avg       0.63      0.61      0.61       154
weighted avg       0.65      0.67      0.66       154

